In [4]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

import os
import random

from tensorflow.keras.experimental import LinearModel
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [38]:
COLUMN_NAMES = ["sepal_length","sepal_width","petal_length","petal_width","species"]

In [6]:
BASE_PATH = f"{os.path.abspath('')}/../data/iris/iris.data"

In [7]:
def preprocessing():
    CLASSES = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]

    new_file_content = []
    with open(BASE_PATH, "r") as f:
        for line in f:
            stripped_line = line.strip()
            stripped_line = stripped_line.split(",")

            if len(stripped_line) != 5:
                continue

            for i in range(len(CLASSES)):
                if stripped_line[4] == CLASSES[i]:
                    stripped_line[4] = str(i)

            stripped_line = ",".join(stripped_line)
            new_file_content.append(stripped_line)

    with open(BASE_PATH, "w") as f:
        random.shuffle(new_file_content)
        new_file_content = "\n".join(new_file_content)
        f.write(new_file_content)

    with open(BASE_PATH, "r") as f:     
        data = np.loadtxt(f, delimiter=",")
        
    return np.hsplit(data, np.array([4]))

In [93]:
def linear_regression(iris_x, iris_y): 

    iris_x = np.array([[i[0]] for i in iris_x])
    
    x = iris_x[:-100]
    iris_x_test = iris_x[-100:]

    y = iris_y[:-100]
    iris_y_test = iris_y[-100:]

    feature_columns = [
        tf.feature_column.numeric_column(key="x")
    ]
    
    input_train = tf.compat.v1.estimator.inputs.numpy_input_fn(
        x={"x": x},
        y=y,
        num_epochs=None,
        shuffle=True)
    
    input_test = tf.compat.v1.estimator.inputs.numpy_input_fn(
        x={"x": iris_x_test},
        y=iris_y_test,
        num_epochs=1,
        shuffle=False)
    
    def accuracy(labels, predictions):
        auc_metric = tf.keras.metrics.CategoricalAccuracy(name="accuracy")
        print(predictions)
        auc_metric.update_state(y_true=labels, y_pred=predictions["predictions"])
        return {'accuracy': auc_metric}
    
    def recall(labels, predictions):
        auc_metric = tf.keras.metrics.Recall(name="recall")
        print(predictions)
        auc_metric.update_state(y_true=labels, y_pred=predictions["predictions"])
        return {'recall': auc_metric}
    
    def precision(labels, predictions):
        auc_metric = tf.keras.metrics.Precision(name="precision")
        print(predictions)
        auc_metric.update_state(y_true=labels, y_pred=predictions["predictions"])
        return {'precision': auc_metric}
    
    model = tf.estimator.LinearRegressor(feature_columns=feature_columns)
    model = tf.estimator.add_metrics(model, accuracy)
    model = tf.estimator.add_metrics(model, recall)
    model = tf.estimator.add_metrics(model, precision)
    model.train(input_fn=input_train, steps=1000)
    eval_result = model.evaluate(input_fn=input_test)
    
    print(eval_result)
    
    return model

In [121]:
def boosted_tree(): 
    dataset = pd.read_csv(BASE_PATH, names=COLUMN_NAMES, header=0)
    dftrain = dataset[:20]
    dfeval = dataset[-20:]
    y_train = dftrain.pop("species")
    y_eval = dfeval.pop("species")
    
    fc = tf.feature_column
#     CATEGORICAL_COLUMNS = ["species"]
    
    NUMERIC_COLUMNS = ["sepal_length","sepal_width","petal_length","petal_width"]
    
    def one_hot_cat_column(feature_name, vocab):
          return fc.indicator_column(
              fc.categorical_column_with_vocabulary_list(feature_name,
                                                         vocab))
    
    feature_columns = []
#     for feature_name in CATEGORICAL_COLUMNS:
#         # Need to one-hot encode categorical features.
#         vocabulary = dftrain[feature_name].unique()
#         feature_columns.append(one_hot_cat_column(feature_name, vocabulary))

    for feature_name in NUMERIC_COLUMNS:
        feature_columns.append(fc.numeric_column(feature_name,
                                               dtype=tf.float32))
    # Use entire batch since this is such a small dataset.
    NUM_EXAMPLES = len(y_train)

    def make_input_fn(X, y, n_epochs=None, shuffle=True):
        def input_fn():
            dataset = tf.data.Dataset.from_tensor_slices((X.to_dict(orient='list'), y))
            if shuffle:
                dataset = dataset.shuffle(NUM_EXAMPLES)
            # For training, cycle thru dataset as many times as need (n_epochs=None).
            dataset = (dataset
              .repeat(n_epochs)
              .batch(NUM_EXAMPLES))
            return dataset
        return input_fn

    # Training and evaluation input functions.
    train_input_fn = make_input_fn(dftrain, y_train)
    eval_input_fn = make_input_fn(dfeval, y_eval, shuffle=False, n_epochs=1)
    
    params = {
      'n_trees': 50,
      'max_depth': 3,
      'n_batches_per_layer': 1,
      # You must enable center_bias = True to get DFCs. This will force the model to
      # make an initial prediction before using any features (e.g. use the mean of
      # the training labels for regression or log odds for classification when
      # using cross entropy loss).
      'n_classes': 3
    }
    
    def accuracy(labels, predictions):
        auc_metric = tf.keras.metrics.CategoricalAccuracy(name="accuracy")
        print(predictions)
        auc_metric.update_state(y_true=labels, y_pred=predictions["class_ids"][0])
        return {'accuracy': auc_metric}
    
    def recall(labels, predictions):
        auc_metric = tf.keras.metrics.Recall(name="recall")
        print(predictions)
        auc_metric.update_state(y_true=labels, y_pred=predictions["class_ids"][0])
        return {'recall': auc_metric}
    
    def precision(labels, predictions):
        auc_metric = tf.keras.metrics.Precision(name="precision")
        print(predictions)
        auc_metric.update_state(y_true=labels, y_pred=predictions["class_ids"][0])
        return {'precision': auc_metric}

    model = tf.estimator.BoostedTreesClassifier(feature_columns, **params)
    model = tf.estimator.add_metrics(model, accuracy)
    model = tf.estimator.add_metrics(model, recall)
    model = tf.estimator.add_metrics(model, precision)
    # Train model.
    model.train(train_input_fn, max_steps=100)

    # Evaluation.
    results = model.evaluate(eval_input_fn)
    print(results)
    pd.Series(results).to_frame()

In [122]:
iris_x, iris_y = preprocessing()

In [123]:
from IPython.display import clear_output
boosted_tree()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/8v/zyfv9h2d2z7fmc3ttvht1mm00000gn/T/tmpev6adi4h', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using config: {'_mode

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.37324s
INFO:tensorflow:Finished evaluation at 2021-05-09-22:51:57
INFO:tensorflow:Saving dict for global step 100: accuracy = 1.0, average_loss = 0.001633613, global_step = 100, loss = 0.001633613, precision = 0.75, recall = 1.0
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: /var/folders/8v/zyfv9h2d2z7fmc3ttvht1mm00000gn/T/tmpev6adi4h/model.ckpt-100
{'accuracy': 1.0, 'average_loss': 0.001633613, 'loss': 0.001633613, 'precision': 0.75, 'recall': 1.0, 'global_step': 100}


In [124]:
linear_regression(iris_x, iris_y)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/8v/zyfv9h2d2z7fmc3ttvht1mm00000gn/T/tmpdubawxq8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using config: {'_mode

In [125]:
boosted_tree()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/8v/zyfv9h2d2z7fmc3ttvht1mm00000gn/T/tmpnb81i2wx', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using config: {'_mode

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.38649s
INFO:tensorflow:Finished evaluation at 2021-05-10-18:22:01
INFO:tensorflow:Saving dict for global step 100: accuracy = 1.0, average_loss = 0.0027466156, global_step = 100, loss = 0.0027466156, precision = 0.75, recall = 1.0
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: /var/folders/8v/zyfv9h2d2z7fmc3ttvht1mm00000gn/T/tmpnb81i2wx/model.ckpt-100
{'accuracy': 1.0, 'average_loss': 0.0027466156, 'loss': 0.0027466156, 'precision': 0.75, 'recall': 1.0, 'global_step': 100}
